In [1]:
import numpy as np
import random
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from data import BirdClefTrainAudio, BirdClefHarmonics, BirdClefClassDS
from model import HarmonicModel
from mimir import training
import time
import pickle
import os

In [2]:
np.random.seed(20250403)
random.seed(20250403)
torch.manual_seed(20250403)
torch.cuda.manual_seed(20250403)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [3]:
data_folder = "data"

In [4]:
audio = BirdClefTrainAudio(data_folder, max_duration=5, sr=16000)
cachefile="data.pkl"
if os.path.isfile(cachefile):
    with open(cachefile, "rb") as f:
        ds = pickle.load(f)
else:
    start = time.time()
    ds = BirdClefHarmonics(audio, fmin=500, fmax=4000)
    print(f"Loading data took {time.time()-start} seconds")
    with open(cachefile, "wb") as f:
        pickle.dump(ds, f)
class_ds = BirdClefClassDS(audio, ds)

In [5]:
loss = nn.CrossEntropyLoss(weight=torch.tensor(audio.class_weights()).to(training.DEVICE))

In [6]:
def accuracy(preds, y):
    return sum(preds.argmax(1) == y)/len(y)

In [7]:
class_hps = training.HyperParameters(model_params={'n_labels': audio.n_classes, 'n_harmonics': 10,
                                                   "num_filter_maps": 2048, "kernel_size": 200, "hidden": 32, "dropout": 0.5}, 
                                     optimizer_params={'lr': 2e-4})

In [8]:
class_results, class_model = training.train(data=class_ds, model_class = HarmonicModel, hyper_params=class_hps, loss_fn = loss,
                                            name="harmonicmodel", pad=True, batch_size=16, metrics={"Accuracy": accuracy})

Epoch   0: Loss=0.55824 val, 0.61058 train Accuracy=0.97304 val 0.96736 train 
Epoch   1: Loss=0.52055 val, 0.58077 train Accuracy=0.97304 val 0.96736 train 
Epoch   2: Loss=0.50478 val, 0.57322 train Accuracy=0.97304 val 0.96736 train 
Epoch   3: Loss=0.50466 val, 0.56719 train Accuracy=0.97304 val 0.96736 train 
Epoch   4: Loss=0.51345 val, 0.57287 train Accuracy=0.97304 val 0.96733 train 
Epoch   5: Loss=0.49642 val, 0.56429 train Accuracy=0.97304 val 0.96736 train 
Epoch   6: Loss=0.51840 val, 0.57709 train Accuracy=0.97304 val 0.96736 train 
Epoch   7: Loss=0.49509 val, 0.56503 train Accuracy=0.97304 val 0.96740 train 
Epoch   8: Loss=0.50849 val, 0.57216 train Accuracy=0.97304 val 0.96733 train 
Epoch   9: Loss=0.50140 val, 0.56320 train Accuracy=0.97304 val 0.96736 train 
Epoch  10: Loss=0.48863 val, 0.55693 train Accuracy=0.97269 val 0.96740 train 
Epoch  11: Loss=0.50169 val, 0.56326 train Accuracy=0.97269 val 0.96733 train 
Epoch  12: Loss=0.48880 val, 0.55752 train Accuracy=